In [1]:
import torch
import torch.nn as nn

In [2]:
inputs = torch.Tensor(1,1,28,28) # 배치, 채널, 높이, 너비

In [6]:
conv1 = nn.Conv2d(1,32,3,padding=1)
conv2 = nn.Conv2d(32,64,3,padding=1)
pool = nn.MaxPool2d(2)

In [9]:
out = conv1(inputs)
out = pool(out)
out = conv2(out)
out = pool(out)
out = out.view(out.size(0), -1)
print(out.shape)

torch.Size([1, 3136])


In [10]:
fc = nn.Linear(3136, 10)
out = fc(out)
print(out.shape)

torch.Size([1, 10])


In [16]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torch.nn.init

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(1)
if device == 'cuda':
    torch.cuda.manual_seed_all(1)

In [13]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                         train=True,
                         download=True,
                         transform=transforms.ToTensor())
mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         download=True,
                         transform=transforms.ToTensor())

In [15]:
data_loader = DataLoader(dataset=mnist_train,
                        batch_size=100,
                        shuffle=True,
                        drop_last=True)

In [49]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1,32,3,1,1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32,64,3,1,1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc = torch.nn.Linear(7*7*64, 10, bias=True)
        
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
        

In [50]:
model = CNN().to(device)

In [65]:
cost = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.1)

In [66]:
total_batch = len(data_loader)
print("총 배치의 수 : {}".format(total_batch))

총 배치의 수 : 600


In [71]:
for epoch in range(11):
    avg_cost = 0
    
    for idx, data in enumerate(data_loader):
        x, y = data
        
        x = x.to(device)
        y = y.to(device)
        
        hypo = model(x)
        loss = cost(hypo, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if idx % 200 == 1 :
            print(f'epoch {epoch}: {idx}/{len(data_loader)} cost {loss.item()}')

epoch 0: 1/600 cost 0.45480412244796753
epoch 0: 201/600 cost 0.36333411931991577
epoch 0: 401/600 cost 0.5497111678123474
epoch 1: 1/600 cost 0.3403093218803406
epoch 1: 201/600 cost 0.7700449228286743
epoch 1: 401/600 cost 0.30430787801742554
epoch 2: 1/600 cost 0.28460681438446045
epoch 2: 201/600 cost 0.2352205216884613
epoch 2: 401/600 cost 0.1281309574842453
epoch 3: 1/600 cost 0.35712066292762756
epoch 3: 201/600 cost 0.2702096104621887
epoch 3: 401/600 cost 0.456085205078125
epoch 4: 1/600 cost 0.1895650327205658
epoch 4: 201/600 cost 0.2953433692455292
epoch 4: 401/600 cost 0.28070932626724243
epoch 5: 1/600 cost 0.3077188730239868
epoch 5: 201/600 cost 0.34438496828079224
epoch 5: 401/600 cost 0.4817807078361511
epoch 6: 1/600 cost 0.37607529759407043
epoch 6: 201/600 cost 0.11265438795089722
epoch 6: 401/600 cost 0.21125268936157227
epoch 7: 1/600 cost 0.19984708726406097
epoch 7: 201/600 cost 0.3572962284088135
epoch 7: 401/600 cost 0.47909897565841675
epoch 8: 1/600 cost 0

In [68]:
len(data_loader)

600

In [73]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
                                       
    pred = model(X_test)
    corr_pred = torch.argmax(pred, 1) == Y_test
    accuracy = corr_pred.float().mean()
    print('accuracy :', accuracy.item())

/home/ubuntu/.local/lib/python3.6/site-packages/torchvision/datasets/mnist.py:69: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


accuracy : 0.8337999582290649


/home/ubuntu/.local/lib/python3.6/site-packages/torchvision/datasets/mnist.py:59: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [ ]:
mnist_train = dsets.mnist(root='mnist_data/',
                         train=True,
                         download=True,
                         transform=transforms.ToTensor())
mnist_test = dsets.mnist(root='mnist_data/',
                        train=True,
                        download=True,
                        transform=transforms.ToTensor())

In [ ]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                         batch_size=100,
                                         shuffle=True,
                                         drop_last=True)